<a href="https://colab.research.google.com/github/KimuYounguWoo/machine-learning-programming/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5-3 트리의 앙상블
## 목표
1. 앙상블 학습에 대한 이해
2. 다양한 앙상블 학습 알고리즘 실습

## 랜덤포레스트
앙상블 학습의 하나로 안정적인 성능으로 여러 방면에서 사용됩니다.
<br>결정 트리를 랜덤하게 만들어, 그 트리들로 '숲'을 만들고, 그 트리들의 예측을 이용해 최종 예측을 만듭니다.

부트스트랩 방식으로 트리를 만드는데, 부트스트랩 방식이란, 데이터를 중복을 허용하여 샘플링합니다.
<br>

또한, 분류는 특성을 특성 개수의 제곱근 만큼 사용하고, 회귀는 전체 특성을 사용합니다.

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [5]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


이전 결정트리보다 중요도가 고루 분포해있는데, 그 이유는 랜덤 포레스트는 특성의 일부를 랜덤하게 선택하여, 훈련하기에 더 많은 특성이 관여할 기회를 줍니다

In [6]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [7]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


OOB는 검증 세트와 비슷하게, 부트스트랩 샘플이 아닌 남는 샘플을 이용하여 점수는 얻어줍니다. 이를 사용하면, 교차 검증을 대신할 수 있어 결과적으로 더 많은 샘플을 훈련에 사용합니다.

### 엑스트라 트리
엑스트라 트리는 램덤 포레스트와 매우 비슷합니다. 대신 결정 트리에서 노드가 분기할 때, 그 노드의 분기를 랜덤으로 합니다. 그렇기에 성능은 낮아지지만, 과대적합을 막고 검증 세트의 점수를 높입니다

훈련 세트는 60, 검증 세트는 20, 테스트 테스틑 20으로 분할해준다.

In [8]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [9]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


특성이 많지 않아 두 모델의 차이가 크지 않지만, 랜덤 분할이기에 속도가 빠릅니다.

### 그레디언트 부스팅
깊이가 얕은 결정 트리를 사용하여, 이전 트리의 오차를 보완하는 방식으로 깊이가 얕기에 과대적합에 강하고, 높은 일반화 성능을 줍니다.

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [12]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

print(np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999
0.8780082549788999


In [13]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


트리의 개수와 학습률을 늘려도, 과대적합에 강하기에 성능 향상의 여지가 존재합니다.

### 히스토그램 기반 그레디언트 부스팅
HGBC는 입력 특성을 256개 구간으로 나누어, 최적의 분할을 빠르게 찾고, 누락된 값을 위한 구간을 나누어 놓아 전처리를 할 필요가 없다는 특징이 있습니다.<BR>
또한 트리의 개수를 지정하는 대신, max_iter로 부스팅 반복 횟수를 지정하여 사용합니다.

In [14]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [15]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


HGBC는 특성 중요도를 계산하기 위해, permutation_importance를 사용합니다. 이 함수는 특성을 하나씩 랜덤하게 섞어 모델의 변화를 관찰하여 어떤 특성이 중요한 지를 계산합니다.
```
n_repeats = 랜덤으로 섞을 횟수
```를 지정합니다

In [16]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


평균 말고도, 표준 표차도 알 수 있습니다.

In [17]:
hgb.score(test_input, test_target)

0.8723076923076923

### HGB와 같은 알고리즘을 가지는 라이브러리
1. XGboost
2. LightGBM

In [18]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [19]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


이 라이브러리들도 위 비슷한 점수를 줍니다.